## Automatic differentiation from scratch

In [1]:
import Base: +, -, *, /, ^, sin, exp

In [10]:
struct AD
    val::Float64
    der::Float64
end

AD(val::Number) = AD(val, 1.0)      # variable


+(a::AD, b::AD) = AD(a.val + b.val, a.der + b.der)
+(a::AD, b::Number) = AD(a.val + b, a.der)
+(a::Number, b::AD) = AD(b.val + b, b.der)

-(a::AD, b::AD) = AD(a.val - b.val, a.der - b.der)
-(a::AD, b::Number) = AD(a.val - b, a.der)
-(a::Number, b::AD) = AD(a - b.val, -b.der)
-(a::AD) = AD(-a.val, -a.der)

*(a::AD, b::AD) = AD(a.val * b.val, a.der * b.val + a.val * b.der)
*(a::AD, b::Number) = AD(a.val * b, a.der * b)
*(a::Number, b::AD) = AD(a * b.val, a * b.der)

sin(a::AD) = AD(sin(a.val), cos(a.val) * a.der)
exp(a::AD) = AD(exp(a.val), exp(a.val) * a.der)

exp (generic function with 14 methods)

In [11]:
x = AD(3)

AD(3.0, 1.0)

In [12]:
x*x+3

AD(12.0, 6.0)

In [21]:
sin(x*x+3)

AD(-0.5365729180004349, 5.063123752394953)

In [14]:
exp(-x*x+3)

AD(0.0024787521766663585, -0.014872513059998151)

## Library

In Julia there is a very good numerical library that already implements automatic differentiation

In [16]:
using ForwardDiff

In [17]:
ForwardDiff.derivative(x -> sin(x^2 + 3), 3)  # should match our result

5.063123752394953

In [18]:
function test(x)
    @show typeof(x)
    @show x
    return sin(x*x + 3)
end

test (generic function with 1 method)

In [20]:
test(3.0)

typeof(x) = Float64
x = 3.0


-0.5365729180004349

ForwardDiff.jl uses Dual number type 

In [22]:
ForwardDiff.derivative(test, 3)

typeof(x) = ForwardDiff.Dual{ForwardDiff.Tag{typeof(test), Int64}, Int64, 1}
x = Dual{ForwardDiff.Tag{typeof(test), Int64}}(3,1)


5.063123752394953

It can compute derivatives, gradients and hessians

In [25]:
const ∂ = ForwardDiff.derivative
const ∇ = ForwardDiff.gradient
const 𝐇 = ForwardDiff.hessian

hessian (generic function with 3 methods)

In [28]:
F(X) = sin(X[1]*X[2]) + X[2]^3
X₀ = [3.0, 4.0]
∇(F, X₀)

2-element Vector{Float64}:
  3.3754158349299686
 50.53156187619748

In [30]:
𝐇(F, X₀)

2×2 Matrix{Float64}:
 8.58517   7.28273
 7.28273  28.8292

### Example 1
Even from some very complicated functions

In [75]:
using QuadGK
∫(f) = quadgk(f, 0, 1)[1]

∫ (generic function with 1 method)

In [76]:
∫(sin)

0.4596976941318603

In [77]:
-cos(1)+1

0.45969769413186023

We calculate a gradient of a function  (calculated _numerically_)
$$G(a) = \int_0^1\frac{dx}{\cosh^{a_2}(a_1x)}$$
with respect to $a_1$ and $a_2$:

$$\nabla G=\left.\begin{pmatrix}
\partial G/\partial{a_1}\\
\partial G/\partial{a_2}
\end{pmatrix}\right|_{a=(1,1)^T}
$$


In [78]:
G(a) = ∫(x->1/cosh(a[1]*x)^a[2])
∇(G, [1.0, 1.0])

2-element Vector{Float64}:
 -0.21771520957577323
 -0.11767856481610509

and the hessian

In [79]:
𝐇(G, X₀)

2×2 Matrix{Float64}:
 0.0492301  0.0103023
 0.0103023  0.0129009

### Example 2
Here we define a function 
$$G(u_0, ω) = u(10)$$
where $u(t)$ is a _numerical_ solution of harmonic oscillator with frequency $\omega$ and initial conditions $u_0=(u(0), u_t(0))$


Then we calculate a gradient with respect to initial conditions and a derivative of $G$ with respect of harmonic oscillator frequency.

In [80]:
using DifferentialEquations


In [81]:
# ODE: u₁' = u₂, u₂' = -ω^2 * u₁
function harmonic!(du, u, p, t)
    du[1] = u[2]
    du[2] = -p^2 * u[1]
end


function G(u0, ω)
    tspan = (0.0, 10.0)
    prob = ODEProblem(harmonic!, u0, tspan, ω)
    return sol = solve(prob, Tsit5(), reltol=1e-9, abstol=1e-9)[end][1]
end


G (generic function with 2 methods)

In [82]:
u0 = [1.0, 0.5]
ω = 1.0
G(u0, 1.0)

-1.1110820841674902

In [83]:
∇(u0->G(u0, ω), u0)

2-element Vector{Float64}:
 -0.8390715285674595
 -0.5440211107754699

In [84]:
∂(ω->G(u0, ω), ω)

1.5168640204004296